# Unified LLM Generation Pipeline

This notebook provides an interface to the unified generation pipeline.
All logic lives in `generation.py` - this notebook is for interactive testing and batch runs.

The pipeline handles both Paleolithic and Holocene people automatically based on the sampled birth year.

In [1]:
import dill
from tqdm import tqdm

from generation import (
    generate_person,
    generate_batch,
    generate_batch_parallel,
    # Individual steps if needed
    generate_geography,
    generate_demographics,
    generate_structured_incidents,
    generate_historical_context,
    generate_name,
    generate_narrative_plan,
    generate_narrative,
    run_pipeline,
    reset_to_stage
)
from llm_utils import GenerationContext, extract_json

from person import sample_year, sample_person, Person

import copy

# Batch Generation

In [2]:
'''# cost prior, $3.62 -> 10.52 = $0.23 per story
test_people = generate_batch_parallel(n=30, model="gpt-5.2", workers=30)'''

'# cost prior, $3.62 -> 10.52 = $0.23 per story\ntest_people = generate_batch_parallel(n=30, model="gpt-5.2", workers=30)'

In [3]:
'''with open('test_examples.pkl', 'wb') as f:
    dill.dump(test_people,f)
%run export.py test_examples.pkl'''

"with open('test_examples.pkl', 'wb') as f:\n    dill.dump(test_people,f)\n%run export.py test_examples.pkl"

In [4]:
with open('test_examples.pkl', 'rb') as f:
    people = dill.load(f)